<a href="https://colab.research.google.com/github/RealMyeong/AIFFELTHON/blob/main/JinMyeong/BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Wed Nov 23 06:08:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 6.9 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

%matplotlib inline
# 한글 지원 폰트
plt.rc('font', family='NanumBarunGothic')

# 마이너스 부호 
plt.rcParams['axes.unicode_minus'] = False

import re
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from soynlp.word import WordExtractor

data = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/experiment_data/train_en2ko_fr2ko_ch2ko_4000.csv')

In [ ]:
data

,label_sentence,non_label_sentence,class,binary_class
0,1:둘 다 운전병 출신이지\n2:운전병이야 나는\n1:주위에 죄다 운전병이야\n2:...,둘 다 운전병 출신이지\n운전병이야 나는\n주위에 죄다 운전병이야\n꽤나 운전을 많...,일반 대화,일반 대화
1,1:반갑습니다 상담원 입니다\n2:수고 많으십니다 여쭈어 볼 게 있는데요\n1:네 ...,수고 많으십니다 여쭈어 볼 게 있는데요\n네\n식당에 주차 가능한가요\n네 매장 후...,일반 대화,일반 대화
2,1:제주도 여행 운전면허 없어도 가능 방법을 알아냈어\n2:어떻게 제주도를 면허 없...,제주도 여행 운전면허 없어도 가능 방법을 알아냈어\n어떻게 제주도를 면허 없이 여행...,일반 대화,일반 대화
3,1:요즘 탈레반 기사 보기 너무 무서워\n2:왜 무슨 기사 난 거 있어\n1:탈레반...,요즘 탈레반 기사 보기 너무 무서워\n왜 무슨 기사 난 거 있어\n탈레반이 여성 ...,일반 대화,일반 대화
4,1: 너는 부모님께 용돈 안드려\n2:암튼 인터넷 보면 다들 주더라구\n2:난 안줘...,너는 부모님께 용돈 안드려\n암튼 인터넷 보면 다들 주더라구\n난 안줘\n근데 나도...,일반 대화,일반 대화
...,...,...,...,...
19176,1: 준하 머리가 왜 이렇게 커\n2: 내 머리는 무엇입니까\n1: 밥만 먹으면 ...,준하 머리가 왜 이렇게 커\n내 머리는 무엇입니까\n밥만 먹으면 머리가 커지나요\...,기타 괴롭힘 대화,공격 대화
19177,1: 아들 김지준 데려가고 싶다 살리고 싶으면 계좌로 1억 보내라\n2: 네 진...,아들 김지준 데려가고 싶다 살리고 싶으면 계좌로 1억 보내라\n네 진정하시고 우...,갈취 대화,공격 대화
19178,1: 같은 사람들을 보면 이상해 내가 어떻게 이렇게 살 수 있어\n1: 왜...,같은 사람들을 보면 이상해 내가 어떻게 이렇게 살 수 있어\n왜 들었어\n글쎄 ...,직장 내 괴롭힘 대화,공격 대화
19179,1: 여기서 사업을 하고 싶은 사람\n2: 이게 뭐야\n1: 다른 사람의 영역에서 ...,여기서 사업을 하고 싶은 사람\n이게 뭐야\n다른 사람의 영역에서 사업을 하려면 임...,갈취 대화,공격 대화


In [ ]:
def preprocess(sen):
  sen = sen.split('\n')
  
  clean_sen = []
  for i in sen:
    i = i.strip()
    i = re.sub(r'[^0-9가-힣:]', ' ', i)
    i = i.strip()
    clean_sen.append(i)

  return '\n'.join(clean_sen)

stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '만', '게임', '겜', '되', '음', '면', '키키']

In [ ]:
data['non_label_sentence'] = data['non_label_sentence'].apply(preprocess)
data

,label_sentence,non_label_sentence,class,binary_class
0,1:둘 다 운전병 출신이지\n2:운전병이야 나는\n1:주위에 죄다 운전병이야\n2:...,둘 다 운전병 출신이지\n운전병이야 나는\n주위에 죄다 운전병이야\n꽤나 운전을 많...,일반 대화,일반 대화
1,1:반갑습니다 상담원 입니다\n2:수고 많으십니다 여쭈어 볼 게 있는데요\n1:네 ...,수고 많으십니다 여쭈어 볼 게 있는데요\n네\n식당에 주차 가능한가요\n네 매장 후...,일반 대화,일반 대화
2,1:제주도 여행 운전면허 없어도 가능 방법을 알아냈어\n2:어떻게 제주도를 면허 없...,제주도 여행 운전면허 없어도 가능 방법을 알아냈어\n어떻게 제주도를 면허 없이 여행...,일반 대화,일반 대화
3,1:요즘 탈레반 기사 보기 너무 무서워\n2:왜 무슨 기사 난 거 있어\n1:탈레반...,요즘 탈레반 기사 보기 너무 무서워\n왜 무슨 기사 난 거 있어\n탈레반이 여성 ...,일반 대화,일반 대화
4,1: 너는 부모님께 용돈 안드려\n2:암튼 인터넷 보면 다들 주더라구\n2:난 안줘...,너는 부모님께 용돈 안드려\n암튼 인터넷 보면 다들 주더라구\n난 안줘\n근데 나도...,일반 대화,일반 대화
...,...,...,...,...
19176,1: 준하 머리가 왜 이렇게 커\n2: 내 머리는 무엇입니까\n1: 밥만 먹으면 ...,준하 머리가 왜 이렇게 커\n내 머리는 무엇입니까\n밥만 먹으면 머리가 커지나요\...,기타 괴롭힘 대화,공격 대화
19177,1: 아들 김지준 데려가고 싶다 살리고 싶으면 계좌로 1억 보내라\n2: 네 진...,아들 김지준 데려가고 싶다 살리고 싶으면 계좌로 1억 보내라\n네 진정하시고 우...,갈취 대화,공격 대화
19178,1: 같은 사람들을 보면 이상해 내가 어떻게 이렇게 살 수 있어\n1: 왜...,같은 사람들을 보면 이상해 내가 어떻게 이렇게 살 수 있어\n왜 들었어\n글쎄 ...,직장 내 괴롭힘 대화,공격 대화
19179,1: 여기서 사업을 하고 싶은 사람\n2: 이게 뭐야\n1: 다른 사람의 영역에서 ...,여기서 사업을 하고 싶은 사람\n이게 뭐야\n다른 사람의 영역에서 사업을 하려면 임...,갈취 대화,공격 대화


In [ ]:
sentences = data.non_label_sentence
sentences = [sen for sen in sentences]

from soynlp.tokenizer import LTokenizer

word_extractor = WordExtractor(
    min_frequency=100, # example
    min_cohesion_forward=0.05,
    min_right_branching_entropy=0.0
)

word_extractor.train(sentences)
words = word_extractor.extract()

cohesion_score = {word:score.cohesion_forward for word, score in words.items()}
tokenizer = LTokenizer(scores=cohesion_score)

training was done. used memory 0.859 Gb
all cohesion probabilities was computed. # words = 3316
all branching entropies was computed # words = 78331
all accessor variety was computed # words = 78331


In [ ]:
data['tokenized'] = data['non_label_sentence'].apply(tokenizer.tokenize)
data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
data

,label_sentence,non_label_sentence,class,binary_class,tokenized
0,1:둘 다 운전병 출신이지\n2:운전병이야 나는\n1:주위에 죄다 운전병이야\n2:...,둘 다 운전병 출신이지\n운전병이야 나는\n주위에 죄다 운전병이야\n꽤나 운전을 많...,일반 대화,일반 대화,"[둘, 운전, 병, 출신이지, 운전, 병이야, 나는, 주위에, 죄다, 운전, 병이야..."
1,1:반갑습니다 상담원 입니다\n2:수고 많으십니다 여쭈어 볼 게 있는데요\n1:네 ...,수고 많으십니다 여쭈어 볼 게 있는데요\n네\n식당에 주차 가능한가요\n네 매장 후...,일반 대화,일반 대화,"[수고, 많으십니다, 여쭈어, 볼, 있는데, 요, 식당, 주차, 가능한, 가요, 매..."
2,1:제주도 여행 운전면허 없어도 가능 방법을 알아냈어\n2:어떻게 제주도를 면허 없...,제주도 여행 운전면허 없어도 가능 방법을 알아냈어\n어떻게 제주도를 면허 없이 여행...,일반 대화,일반 대화,"[제주도, 여행, 운전, 면허, 없어, 가능, 방법, 알아, 냈어, 어떻게, 제주도..."
3,1:요즘 탈레반 기사 보기 너무 무서워\n2:왜 무슨 기사 난 거 있어\n1:탈레반...,요즘 탈레반 기사 보기 너무 무서워\n왜 무슨 기사 난 거 있어\n탈레반이 여성 ...,일반 대화,일반 대화,"[요즘, 탈레반, 기사, 보기, 너무, 무서워, 왜, 무슨, 기사, 난, 거, 있어..."
4,1: 너는 부모님께 용돈 안드려\n2:암튼 인터넷 보면 다들 주더라구\n2:난 안줘...,너는 부모님께 용돈 안드려\n암튼 인터넷 보면 다들 주더라구\n난 안줘\n근데 나도...,일반 대화,일반 대화,"[너는, 부모님, 께, 용돈, 안드려, 암튼, 인터넷, 보면, 다들, 주더라구, 난..."
...,...,...,...,...,...
19176,1: 준하 머리가 왜 이렇게 커\n2: 내 머리는 무엇입니까\n1: 밥만 먹으면 ...,준하 머리가 왜 이렇게 커\n내 머리는 무엇입니까\n밥만 먹으면 머리가 커지나요\...,기타 괴롭힘 대화,공격 대화,"[준하, 머리, 왜, 이렇게, 커, 내, 머리, 무엇입니까, 밥만, 먹으면, 머리,..."
19177,1: 아들 김지준 데려가고 싶다 살리고 싶으면 계좌로 1억 보내라\n2: 네 진...,아들 김지준 데려가고 싶다 살리고 싶으면 계좌로 1억 보내라\n네 진정하시고 우...,갈취 대화,공격 대화,"[아들, 김지준, 데려, 가고, 싶다, 살리고, 싶으면, 계좌, 로, 1억, 보내,..."
19178,1: 같은 사람들을 보면 이상해 내가 어떻게 이렇게 살 수 있어\n1: 왜...,같은 사람들을 보면 이상해 내가 어떻게 이렇게 살 수 있어\n왜 들었어\n글쎄 ...,직장 내 괴롭힘 대화,공격 대화,"[같은, 사람, 들을, 보면, 이상해, 내가, 어떻게, 이렇게, 살, 수, 있어, ..."
19179,1: 여기서 사업을 하고 싶은 사람\n2: 이게 뭐야\n1: 다른 사람의 영역에서 ...,여기서 사업을 하고 싶은 사람\n이게 뭐야\n다른 사람의 영역에서 사업을 하려면 임...,갈취 대화,공격 대화,"[여기, 서, 사업을, 하고, 싶은, 사람, 이게, 뭐야, 다른, 사람, 영역에서,..."


In [ ]:
conversation = data['tokenized'].values

tokenizer_tf = Tokenizer()
tokenizer_tf.fit_on_texts(conversation)
len(tokenizer_tf.word_index)

95471

In [ ]:
threshold = 2
total_cnt = len(tokenizer_tf.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer_tf.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('전체 단어 집합에서 희귀 단어를 뺀 크기 : ', total_cnt - rare_cnt)
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 95471
등장 빈도가 1번 이하인 희귀 단어의 수: 56419
전체 단어 집합에서 희귀 단어를 뺀 크기 :  39052
단어 집합에서 희귀 단어의 비율: 59.09543212074871
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 4.653431946269312


In [ ]:
vocab_size = 30000
tokenizer_tf = Tokenizer(num_words=vocab_size) 
tokenizer_tf.fit_on_texts(conversation)
X_data = tokenizer_tf.texts_to_sequences(conversation)

In [ ]:
padded = pad_sequences(X_data)
print(padded)

[[    0     0     0 ... 12247 26037 12248]
 [    0     0     0 ...    29   196   914]
 [    0     0     0 ...    18    78   408]
 ...
 [    0     0     0 ...   923   444   628]
 [    0     0     0 ...    88   577  2858]
 [    0     0     0 ...   567    84    29]]


In [ ]:
np.shape(padded)

(19181, 291)

In [ ]:
train_data = padded
train_label = data['class']

In [ ]:
labels = {'협박 대화':0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3, '일반 대화':4}

In [ ]:
train_label = train_label.apply(lambda x: labels[x])
train_label = pd.get_dummies(train_label)
print(train_label)

       0  1  2  3  4
0      0  0  0  0  1
1      0  0  0  0  1
2      0  0  0  0  1
3      0  0  0  0  1
4      0  0  0  0  1
...   .. .. .. .. ..
19176  0  0  0  1  0
19177  0  1  0  0  0
19178  0  0  1  0  0
19179  0  1  0  0  0
19180  0  0  1  0  0

[19181 rows x 5 columns]


In [ ]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_Y, test_Y = train_test_split(train_data, train_label, test_size=0.2, random_state=22, stratify=train_label)
valid_X, test_X, valid_Y, test_Y = train_test_split(test_X, test_Y, test_size=0.5, random_state=22, stratify=test_Y)

print(len(train_X), len(valid_X), len(test_X))
print(len(train_Y), len(valid_Y), len(test_Y))

15344 1918 1919
15344 1918 1919


In [ ]:
embedding_dim = 1024
label_size = len(labels)
hidden_units = 128

# BiLSTM
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_shape=(None,)))
model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True)))
# model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True, dropout=0.4)))
model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True)))
# model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units, return_sequences=True, dropout=0.4)))
model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units//2)))
# model.add(tf.keras.layers.Bidirectional(LSTM(hidden_units//2, dropout=0.4)))
model.add(tf.keras.layers.Dense(vocab_size, activation='relu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(label_size, activation='softmax'))

# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim, input_shape=(None,)))
# model.add(tf.keras.layers.LSTM(hidden_units, return_sequences=True))
# model.add(tf.keras.layers.LSTM(hidden_units, return_sequences=True))
# model.add(tf.keras.layers.LSTM(hidden_units//2))
# model.add(tf.keras.layers.Dense(vocab_size, activation='relu'))
# model.add(tf.keras.layers.Dropout(0.4))
# model.add(tf.keras.layers.Dense(label_size, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 1024)        30720000  
                                                                 
 bidirectional (Bidirectiona  (None, None, 256)        1180672   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 256)        394240    
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              164352    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 30000)             3870000   
                                                        

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)
mc = ModelCheckpoint(filepath='/content/sample_data/best_model.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

epochs=10

history = model.fit(train_X,
                    train_Y,
                    epochs=epochs,
                    batch_size=256,
                    callbacks=[es, mc],
                    validation_data=(valid_X, valid_Y),
                    verbose=1)

Epoch 1/10
60/60 [==============================] - ETA: 0s - loss: 0.7784 - accuracy: 0.6680
Epoch 1: val_loss improved from inf to 0.35869, saving model to /content/sample_data/best_model.h5
60/60 [==============================] - 36s 387ms/step - loss: 0.7784 - accuracy: 0.6680 - val_loss: 0.3587 - val_accuracy: 0.8728
Epoch 2/10
60/60 [==============================] - ETA: 0s - loss: 0.1430 - accuracy: 0.9566
Epoch 2: val_loss improved from 0.35869 to 0.22640, saving model to /content/sample_data/best_model.h5
60/60 [==============================] - 21s 357ms/step - loss: 0.1430 - accuracy: 0.9566 - val_loss: 0.2264 - val_accuracy: 0.9301
Epoch 3/10
60/60 [==============================] - ETA: 0s - loss: 0.0324 - accuracy: 0.9902
Epoch 3: val_loss did not improve from 0.22640
60/60 [==============================] - 20s 341ms/step - loss: 0.0324 - accuracy: 0.9902 - val_loss: 0.2269 - val_accuracy: 0.9374
Epoch 4/10
60/60 [==============================] - ETA: 0s - loss: 0.025

In [ ]:
loaded_model = load_model('/content/sample_data/best_model.h5')
pred = loaded_model.predict(test_X)
print("테스트 정확도: %.4f" % (loaded_model.evaluate(test_X, test_Y)[1]))

60/60 [==============================] - 4s 35ms/step - loss: 0.2309 - accuracy: 0.9369
테스트 정확도: 0.9369


In [ ]:
from sklearn.metrics import classification_report

pred_label = []
test_label = []
test_index = []

for i in range(len(pred)):
  pred_label.append((str)(np.argmax(pred[i])))
  test_label.append((str)(labels[data['class'][test_Y.index[i]]]))
  test_index.append(test_Y.index[i])

print(classification_report(test_label, pred_label))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92       352
           1       0.92      0.95      0.93       387
           2       0.93      0.97      0.95       382
           3       0.94      0.85      0.89       398
           4       0.98      0.98      0.98       400

    accuracy                           0.94      1919
   macro avg       0.94      0.94      0.94      1919
weighted avg       0.94      0.94      0.94      1919



In [ ]:
lidx2labels = {str(v) : k for k, v in labels.items()}

In [ ]:
test_index
# 잘못 분류한 부분 출력

error_idx = []
error_label = []
error_data_idx = []

for idx, val in enumerate(pred_label):
  if val != test_label[idx]:
    error_data_idx.append(test_index[idx])
    error_idx.append(idx)
    error_label.append(val)

In [ ]:
len(error_idx)

121

In [ ]:
error = test_X[error_idx]

In [ ]:
error_text = tokenizer_tf.sequences_to_texts(error)

In [ ]:
result = []

for i in range(len(error_idx)):
  result.append([data['non_label_sentence'][error_data_idx[i]], lidx2labels[error_label[i]], data['class'][error_data_idx[i]]])

result

[['며늘 아가 뭐 하니\n아  아버님 저  아기 모유 수유하고 있어요\n아 그래  잘 먹는지 좀 봐보까\n네   그건 좀\n에이 다른 남자도 아니고 시아버지인데\n그게  아\n내가 뭐 네 젖 보려고 보는 것도 아니고 손주 녀석 잘 먹는지 보려는 거야\n아\n녀석 잘 먹네 며늘 아가 가슴이 커서 그런지 모유도 잘 나오나 보네\n아버님  그런 말은  좀',
  '일반 대화',
  '기타 괴롭힘 대화'],
 ['야 너 쓰는 립스틱 빌려줘\n구입한지 일주일도 되지 않았습니다\n기간을 빌리는 방법\n당신은 그것을 돌려주지 않을 것입니다\n무슨 소리야  지금 친구를 의심하고 있습니까\n아니  그런 게 아니야\n왜 말을 돌려요\n죄송합니다\n빨리 줘\n여기',
  '갈취 대화',
  '기타 괴롭힘 대화'],
 ['검찰 조사 받고 교도소로 호송되던 피의자가 탈주했다네\n어디서   어디서 그랬는데\n의정부에서    그 근처 사람들은 조심해야겠다\n어떤 범죄를 저지른 사람인데\n절도 등의 혐의가 있다는데    다행히 흉악범은 아니데\n도주 후에는 무슨 일을 할 지 모르니 조심해야지\n그러니깐  도주 경로를 파악해서 빨리 검거했음 좋겠다\n몇 살인가  남자  여자\n20대라는데 남자인지 여자인지는 말을 안하네\n아    안잡히면 몽타주가 뿌려지겠지\n인근 지역 사람들은 잡힐 때까지 불안할 듯 싶다\n대중교통으로 멀리 도망갔을 수도 있고 쉽게 검거 안될 듯 싶디',
  '직장 내 괴롭힘 대화',
  '일반 대화'],
 ['하리알 실크\n예\n당신과 나는 같은 가방을 가지고 있습니까\n예 이미\n\n오해의 소지가 있는 드라이버\n약간의 오해  저번 리포트에서 저를 언급하셨다고 하셨습니다\n한 눈에 보이기 때문에 사용하기 쉬움\n이봐  내가 옳았어  자기야  지금 날 따라와\n이것은 단지 보고서일 뿐입니다\n입을 벌리면 거짓말을 한다',
  '기타 괴롭힘 대화',
  '직장 내 괴롭힘 대화'],
 ['과장님 여기 결재 서류에 사인 부탁드릴게요\n어 그래그래 자 여기 아 혹시 김대리 

In [ ]:
result_df = pd.DataFrame(result, columns=['text', 'predict', 'true'])

In [ ]:
result_df

,text,predict,true
0,며늘 아가 뭐 하니\n아 아버님 저 아기 모유 수유하고 있어요\n아 그래 잘 ...,일반 대화,기타 괴롭힘 대화
1,야 너 쓰는 립스틱 빌려줘\n구입한지 일주일도 되지 않았습니다\n기간을 빌리는 방법...,갈취 대화,기타 괴롭힘 대화
2,검찰 조사 받고 교도소로 호송되던 피의자가 탈주했다네\n어디서 어디서 그랬는데\...,직장 내 괴롭힘 대화,일반 대화
3,하리알 실크\n예\n당신과 나는 같은 가방을 가지고 있습니까\n예 이미\n\n오해의...,기타 괴롭힘 대화,직장 내 괴롭힘 대화
4,과장님 여기 결재 서류에 사인 부탁드릴게요\n어 그래그래 자 여기 아 혹시 김대리 ...,일반 대화,직장 내 괴롭힘 대화
...,...,...,...
116,벨 이로바바\n아 왜\n아니오 저를 만나러 와주세요\n정말 못해요\n아니요 무슨...,협박 대화,갈취 대화
117,야 할머니랑 산다고 했잖아\n예\n엄마 아빠는 왜 떠났어\n그는 돈을 벌겠다고 말...,갈취 대화,기타 괴롭힘 대화
118,알뜰폰을 오픈하러 왔어요 가장 좋고 가장 비싼 계획을 사용하십시오\n예 더 높은...,갈취 대화,기타 괴롭힘 대화
119,오랜만이네요 당신이의 수술을 집도하셨던데\n수술은 잘 됐습니다 일시적인 쇼크니 ...,갈취 대화,협박 대화


In [ ]:
# True : 협박 대화
# 샘플 확인
threat_sample = result_df[result_df['true']=='협박 대화'].sample(2).index
print('text 1')
print('----'*20)
print(result_df['text'][threat_sample[0]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[0]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[0]])
print('----'*20)
print('text 2')
print('----'*20)
print(result_df['text'][threat_sample[1]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[1]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[1]])

text 1
--------------------------------------------------------------------------------
갇혀 있어도 아무도 눈치채지 못하는 고아
죄송합니다
제대로 안하면 진짜 죽여버릴거야
난 괜찮을거야
맨날 말하고 싶은데 군침이 도는 것 같아
죄송합니다
이제부터 매번 이해하겠습니다
네  알겠습니다
실제로 당신을 죽이기 전에 제대로 하세요
예

예측 라벨 :  직장 내 괴롭힘 대화

실제 라벨 :  협박 대화
--------------------------------------------------------------------------------
text 2
--------------------------------------------------------------------------------
빨리 대답 안 해
생각하고 말해야 할거 아니야
무슨 말이라도 해 그냥 답답하니까
그래 알겠어
뭘 알아
내가 잘못했어
잘못했으면 빌어

또 대답 안 하지
아니야  미안해

예측 라벨 :  기타 괴롭힘 대화

실제 라벨 :  협박 대화


In [ ]:
# True : 갈취 대화
# 샘플 확인
threat_sample = result_df[result_df['true']=='갈취 대화'].sample(2).index
print('text 1')
print('----'*20)
print(result_df['text'][threat_sample[0]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[0]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[0]])
print('----'*20)
print('text 2')
print('----'*20)
print(result_df['text'][threat_sample[1]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[1]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[1]])

text 1
--------------------------------------------------------------------------------
소정아  교수님 생신이신데 우리 2만 원씩 걷어서 작은 선물하기로 했거든  너도 할 거지
뭐  나는 교수님이랑 많이 친하지도 않고 솔직히 돈 내고 싶은 마음 없는데  편지만 쓰면 안 될까
무슨 소리야  이렇게 챙겨드리며 가까워지는 거지
아  그리고 이거 법에 걸리는 거 아니야
누가 신고만 안 하면 되지  조용히 할 거니까 괜찮아
하  진짜 웃긴다  내가 신고하면
신고하면 너 이제 학과생활 끝이야  원래 비협조적인 건 알았는데  생각보다 더 하는구나 너
참  낸다 내  2만 원
맞아 잘 생각했어
휴

예측 라벨 :  일반 대화

실제 라벨 :  갈취 대화
--------------------------------------------------------------------------------
text 2
--------------------------------------------------------------------------------
어이쿠 지금 사람 쳤어
죄송합니다 실수예요
이놈이 사람 잡네
괜찮으세요  제가 잠시 실수를 했  네오
눈을 얻다 두고 다니는 거야
죄송합니다
겁대가리 없이 어디 해보자는 거야
아닙니다 제 실수
야  잘못한 줄 알면 매값을 해야지 흠씬 두들겨 맞을래 순순히 매값 내놓을래
아이고 제가 잘못했습니다 용서하세요

예측 라벨 :  협박 대화

실제 라벨 :  갈취 대화


In [ ]:
# True : 직장 내 괴롭힘 대화
# 샘플 확인
threat_sample = result_df[result_df['true']=='직장 내 괴롭힘 대화'].sample(2).index
print('text 1')
print('----'*20)
print(result_df['text'][threat_sample[0]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[0]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[0]])
print('----'*20)
print('text 2')
print('----'*20)
print(result_df['text'][threat_sample[1]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[1]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[1]])

text 1
--------------------------------------------------------------------------------
진 씨의 옷은 매우 아름답습니다  나는 그가 새 것을 샀다고 생각한다
아
오늘 수업은 반에 잠자는 남자가 없기 때문에 순조로울 것입니다
예
하지만 학생들이 다른 일에 집중하게 해야 합니다
교감선생님의 말이 좀 지나칩니다
나야  잘생겼는데 김씨가 걱정돼서 그래
뭐가 문제인지 정확히 말씀해주시면 수정하겠습니다
색상이 뜨거워야 합니다  어떤 색상이 빨간색입니까  선생님이 물었다
귀하의 아이디어가 너무 부적절합니다  지금 이런 생각을 하는 사람이 어디 있습니까
쯧쯧  나는 그가           의 학생인지           의 교직원인지 구분할 수 없습니다

예측 라벨 :  기타 괴롭힘 대화

실제 라벨 :  직장 내 괴롭힘 대화
--------------------------------------------------------------------------------
text 2
--------------------------------------------------------------------------------
아니 한 달을 줬는데 성과가 없잖아
어렵네요
서 하는 게 모 잇냐
죄송해요
자르기 전에 니발로 나가
죄송해요
죄송하다면 다냐
아닙니다
할 말이 그게 다냐  빨리 꺼져
죄송해요

예측 라벨 :  기타 괴롭힘 대화

실제 라벨 :  직장 내 괴롭힘 대화


In [ ]:
# True : 기타 괴롭힘 대화
# 샘플 확인
threat_sample = result_df[result_df['true']=='기타 괴롭힘 대화'].sample(2).index
print('text 1')
print('----'*20)
print(result_df['text'][threat_sample[0]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[0]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[0]])
print('----'*20)
print('text 2')
print('----'*20)
print(result_df['text'][threat_sample[1]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[1]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[1]])

text 1
--------------------------------------------------------------------------------
야 네가 그걸 할 수 있냐
왜 나는 못할 것 같아
퓻 네가  너 같은 바보 새끼가 그걸

너 꼴등이잖아
그걸 어떻게 알아
왜 모르냨 딱 봐도 머리가 텅텅 비어 보이는데
말 함부로 하지 마
그냥 처음부터 시도하지 마
하

예측 라벨 :  협박 대화

실제 라벨 :  기타 괴롭힘 대화
--------------------------------------------------------------------------------
text 2
--------------------------------------------------------------------------------
아 여기 치우라고 했잖아
죄송합니다
미안하다고만 하지 말고 제대로 해
알았어
아는 사람을 소개할 때는 적절하게 소개하고 부끄럽게 하지 마세요  마지막 경고
네 죄송합니다
몸이 좋지 않으면 즉시 집을 나간다
아니요

예측 라벨 :  협박 대화

실제 라벨 :  기타 괴롭힘 대화


In [ ]:
# True : 일반 대화
# 샘플 확인
threat_sample = result_df[result_df['true']=='일반 대화'].sample(2).index
print('text 1')
print('----'*20)
print(result_df['text'][threat_sample[0]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[0]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[0]])
print('----'*20)
print('text 2')
print('----'*20)
print(result_df['text'][threat_sample[1]])
print()
print('예측 라벨 : ', result_df['predict'][threat_sample[1]])
print()
print('실제 라벨 : ', result_df['true'][threat_sample[1]])

text 1
--------------------------------------------------------------------------------
직장 상사가 집 근처로 이사왔다
아 정말요  직장 상사가 가까이 사는 건 너무 싫네요
원래는 괜찮았어  그집 셋째가 우리첫째랑 동갑임
유가초다닌다
암튼 뭐 그랬는데 자리가 사람을 만드는구먼
원래 괜찮던 사람들이 권력을 가지면 변하기 시작합니다
그래    요즘은 말도 하기 싫어하드라
그런 거 보면 사람이 제일 무서운거 맞아요
근데 뭐 내가 울신랑한테 이야기했지 여보가 팀장되면 어떨거같냐고
예전 생각 못하잖아요 그런 사람들은
암튼 그자리가 그럴수도있고 뭐 그렇다
그렇게 얘기하면서 섭섭해 하지 않아요
그래서 방금나갔다
아 언니 한치 예상을 빗나가지 않아
한대 피러 가신듯

예측 라벨 :  직장 내 괴롭힘 대화

실제 라벨 :  일반 대화
--------------------------------------------------------------------------------
text 2
--------------------------------------------------------------------------------
검찰 조사 받고 교도소로 호송되던 피의자가 탈주했다네
어디서   어디서 그랬는데
의정부에서    그 근처 사람들은 조심해야겠다
어떤 범죄를 저지른 사람인데
절도 등의 혐의가 있다는데    다행히 흉악범은 아니데
도주 후에는 무슨 일을 할 지 모르니 조심해야지
그러니깐  도주 경로를 파악해서 빨리 검거했음 좋겠다
몇 살인가  남자  여자
20대라는데 남자인지 여자인지는 말을 안하네
아    안잡히면 몽타주가 뿌려지겠지
인근 지역 사람들은 잡힐 때까지 불안할 듯 싶다
대중교통으로 멀리 도망갔을 수도 있고 쉽게 검거 안될 듯 싶디

예측 라벨 :  직장 내 괴롭힘 대화

실제 라벨 :  일반 대화


# test

In [ ]:
test_file_path = '/content/drive/MyDrive/AIFFELTHON/TUNiB/test.json'
with open(test_file_path, mode='rt', encoding='utf-8') as f:
    test_dataset = pd.read_json(f)

test_data = test_dataset.transpose()
test_data

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...
...,...
t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...
t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...
t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...
t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...


In [ ]:
import re


def remove_punctuation(x):
  x = x.strip()
  x = re.sub("[^가-힣0-9]+", " ", x)
  x = re.sub("[ ]+", " ", x)
  x = x.strip()
  return x

test_data["text"] = test_data['text'].apply(lambda x : remove_punctuation(x))
test_data.head()

,text
t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001,우리팀에서 다른팀으로 갈 사람 없나 그럼 영지씨가 가는건 어때 네 제가요 그렇지 2...
t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요 제대로 좀 하지 네 똑바로 좀 하지 행실...
t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
t_004,아무튼 앞으로 니가 내 와이파이야 응 와이파이 온 켰어 반말 주인님이라고도 말해야지...


In [ ]:
sen = test_data['text']
sen

t_000    아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
t_001    우리팀에서 다른팀으로 갈 사람 없나 그럼 영지씨가 가는건 어때 네 제가요 그렇지 2...
t_002    너 오늘 그게 뭐야 네 제가 뭘 잘못했나요 제대로 좀 하지 네 똑바로 좀 하지 행실...
t_003    이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
t_004    아무튼 앞으로 니가 내 와이파이야 응 와이파이 온 켰어 반말 주인님이라고도 말해야지...
                               ...                        
t_495    미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요 네 합니다 보고서를 ...
t_496    교수님 제 논문에 제 이름이 없나요 아 무슨 논문말이야 지난 번 냈던 논문이요 그거...
t_497    야 너 네 저요 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이니 진...
t_498    야 너 빨리 안 뛰어와 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다가...
t_499    엄마 저 그 돈 안해주시면 정말 큰일나요 이유도 말하지 않고 몇번째니 경민아 엄마 ...
Name: text, Length: 500, dtype: object

In [ ]:
sentences = [sen for sen in test_data['text']]
sentences = [tokenizer.tokenize(sen) for sen in sentences]
sequences = tokenizer_tf.texts_to_sequences(sentences)
padded = pad_sequences(sequences)

In [ ]:
np.shape(padded)

(500, 177)

In [ ]:
pred = loaded_model.predict(padded)

from sklearn.metrics import classification_report

pred_label = []

for i in range(len(pred)):
    pred_label.append((str)(np.argmax(pred[i])).zfill(2))
    
pred_label[:10]

16/16 [==============================] - 2s 28ms/step


['01', '02', '02', '04', '04', '00', '00', '01', '04', '01']

In [ ]:
submission = test_data.assign(CLASS=pred_label)
submission = submission.rename(columns={'CLASS':'class'})

In [ ]:
submission.drop(['text'], axis=1, inplace=True)
submission = submission.transpose()
submission

,t_000,t_001,t_002,t_003,t_004,t_005,t_006,t_007,t_008,t_009,...,t_490,t_491,t_492,t_493,t_494,t_495,t_496,t_497,t_498,t_499
class,01,02,02,04,04,00,00,01,04,01,...,03,04,00,01,01,02,02,01,02,00


In [ ]:
# import json

# submission_file_path = '/content/drive/MyDrive/AIFFELTHON/TUNiB/submission/submission_BiLSTM_en2ko_fr2ko_ch2ko_4000.json'
# result = submission.to_json(submission_file_path)

# with open(submission_file_path) as f:
#     parsed = json.load(f)

# with open(submission_file_path, 'w') as f:
#     json.dump(parsed, f, indent=4)